In [44]:
#importing libraries

import numpy as np
import cv2, os, warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
from sklearn.datasets import load_iris
from xgboost import XGBClassifier
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import KMeans

In [28]:
#Global variables

PATH_1 = '/content/drive/MyDrive/Colab_Notebooks/20180404_110003.jpg'
PATH_2 = '/content/drive/MyDrive/Colab_Notebooks/test.jpg'

In [29]:
#Determining euclidean distance

#Data conversion
def data_to_array(path):
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  conversion = np.array(img)
  return conversion

img_1 = data_to_array(PATH_1)
img_2 = data_to_array(PATH_2)

distance = euclidean_distances(img_1, img_2)
print(distance)

[[ 239.56836185  834.83710986 1622.79727631 ... 7610.4333648
  7617.07811434 7619.32339516]
 [ 848.08313272  251.01195191 1071.74763821 ... 7580.27360984
  7587.05997077 7588.12618503]
 [1649.83362798 1073.18917251  253.43046384 ... 7543.95493094
  7551.47925906 7550.26032664]
 ...
 [7612.61380867 7572.95021772 7543.87400478 ...  143.86799505
   519.79322812  796.73772347]
 [7626.45664775 7586.12911833 7558.60522848 ...  519.72300315
   141.50971698  470.2818304 ]
 [7622.76832916 7581.61110847 7552.56281007 ...  773.14034948
   471.71283637  149.34188964]]


In [30]:
#Data collection and labelling

data = []
label = []
path = "/content/drive/MyDrive/Colab_Notebooks/Maize/train/"
c = 0
d = 0
IMG_SIZE = 32

for file in os.listdir(path):
    img=cv2.imread(path+file)
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    img=img.astype('float32')

    if file[:3]=='army_worm':
        if c==200: continue
        c+=1
        label.append("army_worm")
    else:
        if d==200: continue
        d+=1
        label.append("cutworm")

    data.append(img)
data = np.array(data)

In [31]:
#Label encoding

data_label = []
for i in label:
    if i=="army_worm": data_label.append(0)
    else: data_label.append(1)
data_label = np.array(data_label)

In [32]:
# Normalization
data = data/255.0

# k-means accept data with less than 3 dimensions
reshaped_data = data.reshape(len(data),-1)
reshaped_data.shape

(160, 3072)

In [45]:
#Custom scorer

# Define multiple metrics
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average='macro'),
           'recall': make_scorer(recall_score, average='macro'),
           'f1-score': make_scorer(f1_score, average='macro')
          }

# Create a XGBClassifier
clf = KMeans(n_clusters=2, random_state=0)

# Evaluate the model using cross-validation and the custom scorer
scores = cross_validate(clf, reshaped_data, data_label, cv=5, scoring=scoring)

# Print the mean scores for each metric
print("Accuracy mean score:", scores['test_accuracy'].mean())
print("Precision mean score:", scores['test_precision'].mean())
print("Recall mean score:", scores['test_recall'].mean())
print("f1-score:", scores['test_f1-score'].mean())

Accuracy mean score: 0.28125
Precision mean score: 0.5
Recall mean score: 0.140625
f1-score: 0.208869236621318
